In [23]:
import matplotlib.pyplot as plt
import numpy as np
import pickle as pkl
import tensorflow as tf

In [24]:
# %load nn17_ex3_main.py
#!/usr/bin/env python3

# Import Data

# plt.close('all')  # if you like

# load dataset

with open('isolet_crop_train.pkl', 'rb') as f:
    train_data = pkl.load(f)

with open('isolet_crop_test.pkl', 'rb') as f:
    test_data = pkl.load(f)

X_train, y_train = train_data
X_test, y_test = test_data


# normalize the data and check the results

# ...

#print(X_train.mean(axis=0))
#print(X_train.var(axis=0))

# split the data sets, etc.

# ...

In [25]:
# Get training Data statistics

# Number of Training examples
n_train = len(X_train)
# Number of Testing examples
n_test = len(X_test)
# Number of Training features
n_TrainFeatures = len(X_train[0])
# Number of Testing features (should be equal to that of training)
n_TestFeatures = len (X_test[0])
# Number of Training classes
n_TrainClasses =  len(np.unique(y_train))
# Number of Testing Classes (should be equal to that of training)
n_TestClasses = len(np.unique(y_test))

print("Number of training examples =", n_train)
print("Number of testing examples =", n_test)
print("....")
print("Number of Training features =", n_TrainFeatures)
print("Number of Testing features =", n_TestFeatures)
print("....")
print("Number of Training Classes =", n_TrainClasses)
print("Number of Testing Classes =", n_TestClasses)

Number of training examples = 6238
Number of testing examples = 1559
....
Number of Training features = 300
Number of Testing features = 300
....
Number of Training Classes = 26
Number of Testing Classes = 26


##  a) Make yourself familiar with the dataset. Normalize each feature to zero mean and unitvariance.

In [26]:
#X_train, y_train = train_data
#X_test, y_test = test_data

X_mean = np.mean(X_train, axis=0)
X_std = np.std(X_train, axis=0)

# Training data normalization
X_train = (X_train - X_mean) / X_std

# Testing data normalization
X_test = (X_test - X_mean) / X_std

# Split Data

In [27]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train,y_train)

from sklearn.model_selection import train_test_split
# First split in 70% -- 30%
X_train2, X_temp, y_train2, y_temp  = train_test_split(X_train, y_train, test_size=0.3, random_state=42)

#Split the 30% into half to get 15% -- 15% (from X_train)for validation and early stop
X_validation, X_stop, y_validation, y_stop = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

n_input = len(X_train2)

In [28]:
print(X_train.shape)
print(y_train.shape)
print(X_train2.shape)
print(y_train2.shape)
print(X_validation.shape)
print(y_validation.shape)
print(X_stop.shape)
print(y_stop.shape)

(6238, 300)
(6238,)
(4366, 300)
(4366,)
(936, 300)
(936,)
(936, 300)
(936,)


## one-out-of-K coding

In [29]:
tf.reset_default_graph()

x = tf.placeholder(tf.float32, (None, 300))
y = tf.placeholder(tf.int32,(None))
one_hot_y = tf.one_hot(y, 26)


# Network

In [30]:
def Train_Classifier(x,n_TrainFeatures, n_TrainClasses):
    
    print("In Traffic Sign Classifier.......................................................................... \n")
    
    n_hidden_layer1 =20 
    
    n_input1 = n_TrainFeatures #300
    n_classes = n_TrainClasses #26
    # n_input1, n_hidden_layer1 n_classes
    
    # Layer 1: Hidden layer
    layer1_W = tf.Variable(tf.truncated_normal([n_input1, n_hidden_layer1]),name='weights_1')
    layer1_b = tf.Variable(tf.zeros(n_hidden_layer1),name='bias_1')
    
    layer1_out = tf.add(tf.matmul(x,layer1_W),layer1_b)
    layer1_out = tf.sigmoid(layer1_out)
    
    # Layer 2: Output layer
    layer2_W = tf.Variable(tf.truncated_normal([n_hidden_layer1, n_classes]),name='weights_2')
    layer2_b = tf.Variable(tf.zeros(n_classes),name='bias_2')
    
    layer2_out = tf.add(tf.matmul(layer1_out, layer2_W), layer2_b)
    #layer2_out = tf.sigmoid(layer2_out)
    
    logits = layer2_out
    
    return logits

In [31]:
rate = 0.1 # learning rate

EPOCHS = 2000
BATCH_SIZE = 40

logits = Train_Classifier(x,n_TrainFeatures, n_TrainClasses)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)

optimizer = tf.train.RMSPropOptimizer(0.001)

#optimizer = tf.train.AdamOptimizer()
#tf.train.GradientDescentOptimizer(learning_rate=rate)

training_operation = optimizer.minimize(loss_operation)

#tf.train.RMSPropOptimizer(rate)
#tf.train.AdamOptimizer()

In Traffic Sign Classifier.......................................................................... 



In [32]:
correct_prediction = tf.equal(tf.argmax(logits,1), tf.argmax(one_hot_y,1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
saver =tf.train.Saver()

def evaluate(X_data, y_data):
    num_examples = len(X_data)
    total_accuracy = 0
    sess = tf.get_default_session()
    for offset in range(0, num_examples, BATCH_SIZE):
        batch_x, batch_y = X_data[offset:offset+BATCH_SIZE], y_data[offset:offset+BATCH_SIZE]
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        total_accuracy += (accuracy * len(batch_x))
    return total_accuracy / num_examples
    

In [ ]:
save_file = './modelTest1.ckpt'
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = len(X_train2)
    
    TempValidat_acc_list = []
    TempTrain_acc_list = []
    TempEarlyStopping_acc_list = []
    
    validat_acc_list = []
    train_acc_list = []
    earlyStoping_acc_list = []
    
    earlyStoppingMax = -1
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        """
        if len(validat_acc_list) > 1:
            if validat_acc_list[-1] < validat_acc_list[-2] - 1e-3:
                print("#############STOP################")
                break
        """
        X_train21, y_train21 = shuffle(X_train2,y_train2)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train21[offset:end], y_train21[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
            
        validation_accuracy = evaluate(X_validation, y_validation)
        training_accuracy = evaluate(X_train2, y_train2)
        earlyStoping_accuracy = evaluate(X_stop,y_stop)
        
        if earlyStoppingMax < earlyStoping_accuracy:
            earlyStoppingMax = earlyStoping_accuracy
            print('Best so far = {} on iteration {}'.format(earlyStoppingMax,i))
            saver.save(sess,save_file)
            #print(sess.run(tempw))
        
        TempValidat_acc_list.append(validation_accuracy)
        TempTrain_acc_list.append(training_accuracy)
        TempEarlyStopping_acc_list.append(earlyStoping_accuracy)
        
        if i%20 == 0:
            validat_acc_list.append(np.mean(TempValidat_acc_list))
            TempValidat_acc_list = []
            
            train_acc_list.append(np.mean(TempTrain_acc_list))
            TempTrain_acc_list = []           
            
            earlyStoping_acc_list.append(np.mean(TempEarlyStopping_acc_list))
            TempEarlyStopping_acc_list = []  
        
            print("EPOCH {} ...".format(i+1))
            print("Validation Accuracy = {:.3f}".format(validation_accuracy))

        
        

    #saver.save(sess, save_file)
    #print("Model saved")

Training...

Best so far = 0.06303418913267107 on iteration 0
EPOCH 1 ...
Validation Accuracy = 0.050
Best so far = 0.11004273682577997 on iteration 1
Best so far = 0.13782051468315798 on iteration 2
Best so far = 0.17841880418296552 on iteration 3
Best so far = 0.21474358999831045 on iteration 4
Best so far = 0.2692307682118864 on iteration 5
Best so far = 0.3076923047630196 on iteration 6
Best so far = 0.3226495737958158 on iteration 7
Best so far = 0.3504273518283143 on iteration 8
Best so far = 0.3707264966180182 on iteration 9
Best so far = 0.3867521363700557 on iteration 10
Best so far = 0.4017094017094017 on iteration 11
Best so far = 0.41666666641194594 on iteration 12
Best so far = 0.4252136731759096 on iteration 14
Best so far = 0.4358974364068773 on iteration 15
Best so far = 0.44230768925104386 on iteration 16
Best so far = 0.44978632223911774 on iteration 17
Best so far = 0.47115384488024264 on iteration 18
Best so far = 0.47863247914192003 on iteration 19
Best so far = 0.

In [ ]:
plt.plot(train_acc_list)
plt.show()

In [ ]:
plt.plot(validat_acc_list)
plt.show()

In [ ]:
plt.plot(validat_acc_list)
plt.show()

In [ ]:
plt.plot(earlyStoping_acc_list)
plt.show()

In [ ]:
plt.plot(earlyStoping_acc_list)
plt.show()

In [ ]:

#tf.reset_default_graph()
n_hidden_layer1 =20 

n_input1 = n_TrainFeatures #300
n_classes = n_TrainClasses #26
# n_input1, n_hidden_layer1 n_classes

tf.reset_default_graph()

layer1_W = tf.Variable(tf.truncated_normal([n_input1, n_hidden_layer1]),name='weights_1')
layer1_b = tf.Variable(tf.zeros(n_hidden_layer1),name='bias_1')


# Layer 2: Output layer
layer2_W = tf.Variable(tf.truncated_normal([n_hidden_layer1, n_classes]),name='weights_2')
layer2_b = tf.Variable(tf.zeros(n_classes),name='bias_2')

saver = tf.train.Saver()
with tf.Session() as sess:

    saver.restore(sess, save_file)
    #evaluate(X_stop,y_stop);
    temp = (sess.run(layer1_W))
print('Loaded Weights successfully.')

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

save_file = './Test2.ckpt'

# Two Tensor Variables: weights and bias

bias = tf.Variable(tf.truncated_normal([3]))
weights = tf.Variable(tf.truncated_normal([2, 3]))

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Save Weights: {}'.format(weights))
print('Save Bias: {}'.format(bias))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, save_file)

# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
bias = tf.Variable(tf.truncated_normal([3]))
weights = tf.Variable(tf.truncated_normal([2, 3]))

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Load Weights: {}'.format(weights))
print('Load Bias: {}'.format(bias))

with tf.Session() as sess:
    # Load the weights and bias - No Error
    saver.restore(sess, save_file)

print('Loaded Weights and Bias successfully.')

In [ ]:
import tensorflow as tf

tf.reset_default_graph()

save_file = './Test.ckpt'

# Two Tensor Variables: weights and bias
weights = tf.Variable(tf.truncated_normal([2, 3]), name='weights_0')
bias = tf.Variable(tf.truncated_normal([3]), name='bias_0')

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Save Weights: {}'.format(weights.name))
print('Save Bias: {}'.format(bias.name))

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    saver.save(sess, save_file)

# Remove the previous weights and bias
tf.reset_default_graph()

# Two Variables: weights and bias
bias = tf.Variable(tf.truncated_normal([3]), name='bias_0')
weights = tf.Variable(tf.truncated_normal([2, 3]) ,name='weights_0')

saver = tf.train.Saver()

# Print the name of Weights and Bias
print('Load Weights: {}'.format(weights.name))
print('Load Bias: {}'.format(bias.name))

with tf.Session() as sess:
    # Load the weights and bias - No Error
    saver.restore(sess, save_file)

print('Loaded Weights and Bias successfully.')